In [1]:
import pandas as pd
import numpy as np
import altair as alt
import statsmodels.formula.api as smf
import altair_saver
from find_control_module import find_controls

alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [2]:
florida = pd.read_parquet(f"../20_intermediate_files/florida_shipment_monthly.parquet")
controls = pd.read_parquet(f"../20_intermediate_files/other_shipment_monthly.parquet")


In [3]:
controls.loc[:, "seconds"] = pd.to_datetime(controls["year"]).astype(int) / 10 ** 9
florida.loc[:, "seconds"] = pd.to_datetime(florida["year"]).astype(int) / 10 ** 9


/var/folders/lz/cw17t6h96s12fy8v30l0cs880000gn/T/ipykernel_98885/3401772982.py:1: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  controls.loc[:, "seconds"] = pd.to_datetime(controls["year"]).astype(int) / 10 ** 9
/var/folders/lz/cw17t6h96s12fy8v30l0cs880000gn/T/ipykernel_98885/3401772982.py:2: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  florida.loc[:, "seconds"] = pd.to_datetime(florida["year"]).astype(int) / 10 ** 9


In [4]:
control_florida = controls.copy()
control_florida["category_flo"] = np.where(
    control_florida["year"] >= pd.to_datetime("2010-02-01"), "post", "pre"
)
control_florida["type"] = "Control States"
florida["type"] = "Florida"
florida_pre = florida[florida["category_flo"] == "pre"]
florida_post = florida[florida["category_flo"] == "post"]
control_florida_pre = control_florida[control_florida["category_flo"] == "pre"]
control_florida_post = control_florida[control_florida["category_flo"] == "post"]


In [5]:
selected_control_data = find_controls(
    "shipment", florida_pre, control_florida_pre, control_florida, n=2
)

selected_control_pre = selected_control_data[
    selected_control_data["category_flo"] == "pre"
]
selected_control_post = selected_control_data[
    selected_control_data["category_flo"] == "post"
]
pre = pd.concat([florida_pre, selected_control_pre])
post = pd.concat([florida_post, selected_control_post])


/Users/Razalamb1/MIDS/fall21/practicing/pds2021-opioids-team-9-1/10_code/find_control_module.py:34: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  df_pre.loc[:, "seconds"] = pd.to_datetime(df_pre["year"]).array.astype(int) / 10 ** 9
/Users/Razalamb1/opt/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/Razalamb1/MIDS/fall21/practicing/pds2021-opioids-team-9-1/10_code/find_control_module.py:34: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) 

In [6]:
def get_reg_fit(data, yvar, xvar, color, alpha=0.05):
    # Grid for predicted values
    x = data.loc[pd.notnull(data[yvar]), xvar]
    grid = np.asarray(data[xvar].unique())
    predictions = pd.DataFrame({xvar: grid})
    # Fit model, get predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)
    predictions[xvar] = pd.to_datetime(10 ** 9 * predictions[xvar])
    return predictions


In [7]:
flo1 = get_reg_fit(florida[florida['category_flo'] == "pre"], 'shipment_per_resident', 'seconds', 'orange')
flo2 = get_reg_fit(florida[florida['category_flo'] == "post"], 'shipment_per_resident', 'seconds', 'orange')
flo = pd.concat([flo1, flo2])


In [16]:
flo['category_flo'] = np.where(
    pd.to_datetime(flo["seconds"]) >= pd.to_datetime("2010-02-01"), "post", "pre"
)
reg_flo = (
    alt.Chart(
        flo, title="Opioid Shipments per County per Month in Florida"
    )
    .mark_line(color = "orange")
    .encode(
        x=alt.X("seconds:T", scale=alt.Scale(zero=False), title="Year"),
        y=alt.Y(
            "shipment_per_resident",
            scale=alt.Scale(zero=False),
            title="Opioid Shipment Per Resident (MME)",
        ),
        detail="category_flo",
    )
)
ci_flo = (
    alt.Chart(flo)
    .mark_errorband(color="orange")
    .encode(x=f"seconds:T", y=alt.Y("ci_low", title=""), y2="ci_high", detail = "category_flo")
)
data = pd.DataFrame({"a": ["2010-02-01"]})
sep_line = alt.Chart(data).mark_rule(color="black", strokeDash=[10, 10]).encode(x="a:T")

pre_post = ((reg_flo + ci_flo + sep_line)
    .properties(height=500, width=1000)
    .configure_axis(labelFontSize=20, titleFontSize=20)
    .configure_title(fontSize=30)
    .configure_legend(titleFontSize=20, labelFontSize=18)
)
altair_saver.save(pre_post, f"../30_results/prepost_Florida_shipment_monthly.png")


WARN x-scale's "zero" is dropped as it does not work with time scale.


In [9]:
predictions = get_reg_fit(
    pre[pre["type"] == "Florida"], "shipment_per_resident", "seconds", "orange"
)
predictions["type"] = "Florida"
predictions2 = get_reg_fit(
    pre[pre["type"] == "Control States"], "shipment_per_resident", "seconds", "blue"
)
predictions2["type"] = "Control Counties"



In [10]:
pred_final = pd.concat([predictions, predictions2])
reg1 = (
    alt.Chart(
        pred_final, title="Opioid Shipments per County per Month, Florida vs. Controls"
    )
    .mark_line()
    .encode(
        x=alt.X("seconds:T", scale=alt.Scale(zero=False), title="Year"),
        y=alt.Y(
            "shipment_per_resident",
            scale=alt.Scale(zero=False),
            title="Opioid Shipment Per Resident (MME)",
        ),
        color="type",
    )
)
alt.X("year:O", scale=alt.Scale(zero=False), title="Year")
ci1flo = (
    alt.Chart(pred_final[pred_final["type"] == "Florida"])
    .mark_errorband(color="orange")
    .encode(x=f"seconds:T", y=alt.Y("ci_low", title=""), y2="ci_high")
)
ci1cont = (
    alt.Chart(pred_final[pred_final["type"] == "Control Counties"])
    .mark_errorband(color="blue")
    .encode(x=f"seconds:T", y=alt.Y("ci_low", title=""), y2="ci_high")
)
chart = ci1flo + ci1cont + reg1


In [11]:
ppredictions = get_reg_fit(
    post[post["type"] == "Florida"], "shipment_per_resident", "seconds", "orange"
)
ppredictions["type"] = "Florida"
ppredictions2 = get_reg_fit(
    post[post["type"] == "Control States"], "shipment_per_resident", "seconds", "blue"
)
ppredictions2["type"] = "Control Counties"
ppred_final = pd.concat([ppredictions, ppredictions2])


In [12]:
reg2 = (
    alt.Chart(ppred_final)
    .mark_line()
    .encode(x="seconds:T", y="shipment_per_resident", color="type")
)
ci2flo = (
    alt.Chart(ppred_final[ppred_final["type"] == "Florida"])
    .mark_errorband(color="orange")
    .encode(
        x=f"seconds:T",
        y=alt.Y("ci_low", title=""),
        y2="ci_high",
    )
)
ci2cont = (
    alt.Chart(ppred_final[ppred_final["type"] == "Control Counties"])
    .mark_errorband(color="blue")
    .encode(
        x=f"seconds:T",
        y=alt.Y("ci_low", title=""),
        y2="ci_high",
    )
)

chart2 = ci2flo + reg2 + ci2cont + sep_line


In [13]:
final_chart = (
    (chart + chart2)
    .properties(height=500, width=1000)
    .configure_axis(labelFontSize=20, titleFontSize=20)
    .configure_title(fontSize=30)
    .configure_legend(titleFontSize=20, labelFontSize=18)
)

altair_saver.save(final_chart, f"../30_results/florida_shipment_monthly.png")


WARN x-scale's "zero" is dropped as it does not work with time scale.


In [14]:
final_chart.display()


alt.LayerChart(...)

In [15]:
ci1flo

alt.Chart(...)